In [2]:
import os, sys, time, tqdm
import numpy as np
import pandas as pd
import scipy.stats as ss

In [3]:
cytoBand = pd.read_csv('../ref/hg19_cytoBand.txt', sep='\t', header=None)
cytoBand.head()

,0,1,2,3,4
0,chr1,0,2300000,p36.33,gneg
1,chr1,2300000,5400000,p36.32,gpos25
2,chr1,5400000,7200000,p36.31,gneg
3,chr1,7200000,9200000,p36.23,gpos25
4,chr1,9200000,12700000,p36.22,gneg


In [4]:
ukb_variants = pd.read_csv('../blood_coloc/gwas/ukb.variants.tsv.bgz', compression='gzip', sep='\t')
ukb_variants.shape

/tmp/ipykernel_988886/3348586129.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  ukb_variants = pd.read_csv('../blood_coloc/gwas/ukb.variants.tsv.bgz', compression='gzip', sep='\t')


(13791467, 25)

# RA

## GWAS preprocess

In [97]:
raw = pd.read_csv('../blood_coloc/gwas/RA-10-28-2021/EUR_all_auto-10-2021.txt.gz', sep=' ')

In [102]:
ref = pd.read_csv('../ref/reference.1000G.maf.0.005.txt', sep=' ')

In [103]:
ref.head()

,SNP,CHR,BP,MAF,A1,A2
0,rs201725126,1,13116,0.186879,T,G
1,rs200579949,1,13118,0.186879,A,G
2,rs75454623,1,14930,0.479125,A,G
3,rs199856693,1,14933,0.050696,G,A
4,rs78601809,1,15211,0.268390,T,G


In [104]:
raw[['CHR', 'BP', 'ref', 'alt']] = raw.SNP.str.split('_', expand=True)

In [105]:
raw.head()

,SNP,Beta,SE,Pval,CHR,BP,ref,alt
0,10_26354101_G_T,-0.0186,0.0151,0.2200,10,26354101,G,T
1,3_16794282_CTGTT_C,-0.0079,0.0140,0.5711,3,16794282,CTGTT,C
2,13_38855689_T_C,-0.0517,0.0828,0.5325,13,38855689,T,C
3,4_137301978_T_C,1.3959,0.5448,0.0104,4,137301978,T,C
4,18_4865341_T_A,0.1558,0.1372,0.2561,18,4865341,T,A


In [110]:
raw = raw.astype({'CHR': int, 'BP': int})
raw.dtypes

SNP      object
Beta    float64
SE      float64
Pval    float64
CHR       int64
BP        int64
ref      object
alt      object
dtype: object

In [111]:
merged = pd.merge(raw, ref, on=['CHR', 'BP'], how='inner')

In [113]:
merged.shape, raw.shape, ref.shape

((8919821, 12), (13297690, 8), (9667224, 6))

In [114]:
merged.head()

,SNP_x,Beta,SE,Pval,CHR,BP,ref,alt,SNP_y,MAF,A1,A2
0,10_26354101_G_T,-0.0186,0.0151,0.2200,10,26354101,G,T,rs10828946,0.438370,G,T
1,13_38855689_T_C,-0.0517,0.0828,0.5325,13,38855689,T,C,rs12017161,0.008946,T,C
2,9_100158876_T_G,-0.0076,0.0154,0.6217,9,100158876,T,G,rs4742689,0.381710,T,G
3,3_141430083_G_A,-0.0094,0.0795,0.9061,3,141430083,G,A,rs16851691,0.009940,G,A
4,19_38452535_C_T,-0.0128,0.0203,0.5290,19,38452535,C,T,rs353416,0.172962,C,T


In [115]:
merged2 = merged.loc[((merged.ref==merged.A1) & (merged.alt==merged.A2)) | ((merged.ref==merged.A2) & (merged.alt==merged.A1))]
merged2.shape

(8919817, 12)

In [134]:
merged3 = merged2[['CHR', 'BP', 'SNP_y', 'alt', 'ref', 'Beta', 'SE', 'Pval']]
merged3.rename(columns={'SNP_y': 'SNP'}, inplace=True)
merged3.head()

/tmp/ipykernel_3686288/282468429.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged3.rename(columns={'SNP_y': 'SNP'}, inplace=True)


,CHR,BP,SNP,alt,ref,Beta,SE,Pval
0,10,26354101,rs10828946,T,G,-0.0186,0.0151,0.2200
1,13,38855689,rs12017161,C,T,-0.0517,0.0828,0.5325
2,9,100158876,rs4742689,G,T,-0.0076,0.0154,0.6217
3,3,141430083,rs16851691,A,G,-0.0094,0.0795,0.9061
4,19,38452535,rs353416,T,C,-0.0128,0.0203,0.5290


In [143]:
merged3 = merged3.sort_values(by=['CHR', 'BP']).reset_index(drop=True)
merged3.head()

,CHR,BP,SNP,alt,ref,Beta,SE,Pval
0,1,751343,rs202152658,A,T,-0.0588,0.0380,0.1220
1,1,751756,rs143225517,C,T,-0.0585,0.0379,0.1230
2,1,752566,rs3094315,A,G,0.0472,0.0328,0.1495
3,1,752721,rs3131972,G,A,0.0325,0.0274,0.2354
4,1,752894,rs3131971,C,T,0.0522,0.0357,0.1439


In [144]:
merged3.shape

(8919817, 8)

In [145]:
merged3.to_csv('../blood_coloc/gwas/RA-10-28-2021/EUR_all_auto-10-2021_processed.txt.gz', compression='gzip', sep='\t', index=False)

## Get index snps

In [146]:
raw = pd.read_csv('../blood_coloc/gwas/RA-10-28-2021/EUR_all_auto-10-2021_processed.txt.gz', sep='\t')

In [147]:
raw.shape

(8919817, 8)

In [148]:
raw.head()

,CHR,BP,SNP,alt,ref,Beta,SE,Pval
0,1,751343,rs202152658,A,T,-0.0588,0.0380,0.1220
1,1,751756,rs143225517,C,T,-0.0585,0.0379,0.1230
2,1,752566,rs3094315,A,G,0.0472,0.0328,0.1495
3,1,752721,rs3131972,G,A,0.0325,0.0274,0.2354
4,1,752894,rs3131971,C,T,0.0522,0.0357,0.1439


In [149]:
pval_thres = 5e-8
sig = raw.loc[raw['Pval']<pval_thres].reset_index(drop=True)
print(sig.shape)

(29823, 8)


In [155]:
sumstat = sig.copy()
sumstat['cyto'] = None
for i, row in tqdm.tqdm(sumstat.iterrows(), total=sumstat.shape[0]):
    tmp = cytoBand.loc[(cytoBand[0]==f'chr{row.CHR}') & (cytoBand[1]<=row.BP) & (cytoBand[2]>=row.BP)]
    sumstat.iloc[i,8] = f'{row.CHR}{tmp[3].values[0]}'

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29823/29823 [00:28<00:00, 1032.76it/s]


In [157]:
index_df = sumstat.loc[sumstat.groupby('cyto')['Pval'].idxmin()].sort_values(by=['CHR', 'BP']).reset_index(drop=True)
index_df = index_df.loc[index_df['SNP'].str.startswith('rs')]
print(index_df.shape)
index_df.to_csv('../blood_coloc/gwas/RA-10-28-2021/EUR_all_auto_indexsnp.csv', sep='\t', index=False)

(69, 9)


In [156]:
sumstat.head()

,CHR,BP,SNP,alt,ref,Beta,SE,Pval,cyto
0,1,2483961,rs2258734,A,G,-0.0939,0.0148,2.521000e-10,1p36.32
1,1,2501222,rs10797431,T,G,-0.0852,0.0138,7.040000e-10,1p36.32
2,1,2516781,rs60733400,A,G,-0.0880,0.0136,1.127000e-10,1p36.32
3,1,2520302,rs6667564,C,T,-0.0844,0.0134,3.228000e-10,1p36.32
4,1,2520500,rs6701238,A,G,-0.0843,0.0134,3.307000e-10,1p36.32


# MS

## GWAS preprocess

In [158]:
raw = pd.read_csv('../blood_coloc/gwas/MS_meta', sep=' ')

In [159]:
raw.shape

(8589719, 8)

In [166]:
raw.head(5)

,CHR,BP,SNP,effect_allele,other_allele,N,P,OR
0,1,11154,chr1:11154,C,A,4,0.7911,0.9818
1,1,11565,chr1:11565,G,T,5,0.8735,0.9924
2,1,11710,chr1:11710,T,G,4,0.7793,0.9806
3,1,11961,chr1:11961,A,G,4,0.7953,0.9817
4,1,12078,chr1:12078,G,A,1,NaN,NaN


In [160]:
ref.head()

,SNP,CHR,BP,MAF,A1,A2
0,rs201725126,1,13116,0.186879,T,G
1,rs200579949,1,13118,0.186879,A,G
2,rs75454623,1,14930,0.479125,A,G
3,rs199856693,1,14933,0.050696,G,A
4,rs78601809,1,15211,0.268390,T,G


In [165]:
raw.rename(columns={'A1': 'effect_allele', 'A2': 'other_allele'}, inplace=True)

In [167]:
merged = pd.merge(raw, ref, on=['CHR', 'BP'], how='inner')
merged.shape, raw.shape, ref.shape

((7959516, 12), (8589719, 8), (9667224, 6))

In [169]:
merged = merged.loc[((merged.effect_allele==merged.A1) & (merged.other_allele==merged.A2)) | ((merged.effect_allele==merged.A2) & (merged.other_allele==merged.A1))]
merged.shape

(7959422, 12)

In [170]:
merged.dropna(subset=['P', 'OR'], inplace=True)
merged.shape

(7753386, 12)

In [193]:
merged['beta'] = np.log(merged.OR)
merged['se'] = np.abs(merged['beta'] / ss.norm.ppf(merged.P/2))

In [197]:
merged2 = merged[['CHR', 'BP', 'SNP_y', 'effect_allele', 'other_allele', 'beta', 'se', 'OR', 'P', 'N']].sort_values(by=['CHR', 'BP']).reset_index(drop=True)
merged2.shape

(7753386, 10)

In [203]:
merged2.rename(columns={'SNP_y': 'SNP'}, inplace=True)

In [206]:
merged2.to_csv('../blood_coloc/gwas/MS_meta_processed', sep='\t', index=False)

## Get index snps

In [207]:
raw = pd.read_csv('../blood_coloc/gwas/MS_meta_processed', sep='\t')
raw.shape

(7753386, 10)

In [208]:
pval_thres = 5e-8
sig = raw.loc[raw['P']<pval_thres].reset_index(drop=True)
print(sig.shape)

(23262, 10)


In [209]:
sig.head()

,CHR,BP,SNP,effect_allele,other_allele,beta,se,OR,P,N
0,1,2470848,rs12042279,A,G,-0.099158,0.018130,0.9056,4.518000e-08,15
1,1,2472081,rs1886731,C,T,-0.105583,0.018366,0.8998,8.982000e-09,15
2,1,2472144,rs4487972,C,G,-0.099378,0.018091,0.9054,3.944000e-08,15
3,1,2474205,rs942824,C,T,-0.100815,0.018060,0.9041,2.374000e-08,15
4,1,2475212,rs10797429,C,G,-0.101037,0.018050,0.9039,2.172000e-08,15


In [212]:
sumstat = sig.copy()
sumstat['cyto'] = None
for i, row in tqdm.tqdm(sumstat.iterrows(), total=sumstat.shape[0]):
    tmp = cytoBand.loc[(cytoBand[0]==f'chr{row.CHR}') & (cytoBand[1]<=row.BP) & (cytoBand[2]>=row.BP)]
    sumstat.iloc[i,10] = f'{row.CHR}{tmp[3].values[0]}'

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 23262/23262 [00:22<00:00, 1014.44it/s]


In [214]:
index_df = sumstat.loc[sumstat.groupby('cyto')['P'].idxmin()].sort_values(by=['CHR', 'BP']).reset_index(drop=True)
index_df = index_df.loc[index_df['SNP'].str.startswith('rs')]
print(index_df.shape)
index_df.to_csv('../blood_coloc/gwas/MS_meta_indexsnp.csv', sep='\t', index=False)

(77, 11)


# Asthma

## GWAS preprocess

In [215]:
raw = pd.read_csv('../blood_coloc/gwas/20002_1111.gwas.imputed_v3.both_sexes.tsv.bgz', compression='gzip', sep='\t')
raw.shape

(13791467, 12)

In [216]:
raw.head()

,variant,minor_allele,minor_AF,expected_case_minor_AC,low_confidence_variant,n_complete_samples,AC,ytx,beta,se,tstat,pval
0,1:15791:C:T,T,5.429410e-09,0.000455,True,361141,0.003922,0.003922,224.209000,81.598800,2.747710,0.006002
1,1:69487:G:A,A,5.749750e-06,0.482220,True,361141,4.152940,0.011765,-0.115130,0.159834,-0.720309,0.471335
2,1:69569:T:C,C,1.877650e-04,15.747500,True,361141,135.620000,15.419600,-0.002088,0.028666,-0.072850,0.941926
3,1:139853:C:T,T,5.662880e-06,0.474934,True,361141,4.090200,0.000000,-0.116331,0.159839,-0.727802,0.466735
4,1:692794:CA:C,C,1.106370e-01,9278.860000,False,361141,79910.800000,9184.790000,-0.001472,0.001324,-1.111400,0.266398


In [ ]:
raw[['CHR', 'BP', 'ref', 'alt']] = raw.SNP.str.split('_', expand=True)

In [228]:
merged = pd.merge(raw, ukb_variants[['variant', 'chr', 'pos', 'ref', 'alt', 'rsid']], on='variant', how='inner')
merged.shape

(13791467, 17)

In [229]:
merged.head()

,variant,minor_allele,minor_AF,expected_case_minor_AC,low_confidence_variant,n_complete_samples,AC,ytx,beta,se,tstat,pval,chr,pos,ref,alt,rsid
0,1:15791:C:T,T,5.429410e-09,0.000455,True,361141,0.003922,0.003922,224.209000,81.598800,2.747710,0.006002,1,15791,C,T,rs547522712
1,1:69487:G:A,A,5.749750e-06,0.482220,True,361141,4.152940,0.011765,-0.115130,0.159834,-0.720309,0.471335,1,69487,G,A,rs568226429
2,1:69569:T:C,C,1.877650e-04,15.747500,True,361141,135.620000,15.419600,-0.002088,0.028666,-0.072850,0.941926,1,69569,T,C,rs2531267
3,1:139853:C:T,T,5.662880e-06,0.474934,True,361141,4.090200,0.000000,-0.116331,0.159839,-0.727802,0.466735,1,139853,C,T,rs533633326
4,1:692794:CA:C,C,1.106370e-01,9278.860000,False,361141,79910.800000,9184.790000,-0.001472,0.001324,-1.111400,0.266398,1,692794,CA,C,1:692794_CA_C


In [230]:
merged = merged.loc[merged.chr.isin(np.arange(1,23))].astype({'chr': int})
print(merged.shape)
merged = merged.loc[merged.low_confidence_variant==False].sort_values(by=['chr', 'pos']).reset_index(drop=True)
merged.shape

(13336576, 17)


(13135953, 17)

In [231]:
merged.head()

,variant,minor_allele,minor_AF,expected_case_minor_AC,low_confidence_variant,n_complete_samples,AC,ytx,beta,se,tstat,pval,chr,pos,ref,alt,rsid
0,1:692794:CA:C,C,0.110637,9278.860,False,361141,79910.80,9184.790,-0.001472,0.001324,-1.111400,0.266398,1,692794,CA,C,1:692794_CA_C
1,1:693731:A:G,G,0.115823,9713.840,False,361141,83656.80,9665.270,-0.000659,0.001251,-0.526387,0.598620,1,693731,A,G,rs12238997
2,1:707522:G:C,C,0.097298,8160.220,False,361141,70276.90,8086.070,-0.001301,0.001406,-0.925050,0.354940,1,707522,G,C,rs371890604
3,1:717587:G:A,A,0.015686,1315.590,False,361141,11330.00,1310.450,-0.000595,0.003356,-0.177415,0.859182,1,717587,G,A,rs144155419
4,1:723329:A:T,T,0.001733,145.379,False,361141,1252.02,157.067,0.011290,0.009901,1.140270,0.254175,1,723329,A,T,rs189787166


In [233]:
merged = merged.loc[merged['rsid'].str.startswith('rs')]
merged.shape

(12556648, 17)

In [234]:
merged.head()

,variant,minor_allele,minor_AF,expected_case_minor_AC,low_confidence_variant,n_complete_samples,AC,ytx,beta,se,tstat,pval,chr,pos,ref,alt,rsid
1,1:693731:A:G,G,0.115823,9713.840,False,361141,83656.80,9665.270,-0.000659,0.001251,-0.526387,0.598620,1,693731,A,G,rs12238997
2,1:707522:G:C,C,0.097298,8160.220,False,361141,70276.90,8086.070,-0.001301,0.001406,-0.925050,0.354940,1,707522,G,C,rs371890604
3,1:717587:G:A,A,0.015686,1315.590,False,361141,11330.00,1310.450,-0.000595,0.003356,-0.177415,0.859182,1,717587,G,A,rs144155419
4,1:723329:A:T,T,0.001733,145.379,False,361141,1252.02,157.067,0.011290,0.009901,1.140270,0.254175,1,723329,A,T,rs189787166
5,1:730087:T:C,C,0.056457,4734.970,False,361141,40778.20,4673.710,-0.001722,0.001743,-0.988276,0.323018,1,730087,T,C,rs148120343


In [235]:
merged[['rsid', 'chr', 'pos', 'ref', 'alt', 'minor_AF', 'beta', 'se', 'tstat', 'pval']].to_csv('../blood_coloc/gwas/Asthma_ukb.csv', sep='\t', index=False)


## Get index snps

In [236]:
raw = pd.read_csv('../blood_coloc/gwas/Asthma_ukb.csv', sep='\t')
raw.shape

(12556648, 10)

In [237]:
raw.head()

,rsid,chr,pos,ref,alt,minor_AF,beta,se,tstat,pval
0,rs12238997,1,693731,A,G,0.115823,-0.000659,0.001251,-0.526387,0.598620
1,rs371890604,1,707522,G,C,0.097298,-0.001301,0.001406,-0.925050,0.354940
2,rs144155419,1,717587,G,A,0.015686,-0.000595,0.003356,-0.177415,0.859182
3,rs189787166,1,723329,A,T,0.001733,0.011290,0.009901,1.140270,0.254175
4,rs148120343,1,730087,T,C,0.056457,-0.001722,0.001743,-0.988276,0.323018


In [238]:
pval_thres = 5e-8
sig = raw.loc[raw['pval']<pval_thres].reset_index(drop=True)
print(sig.shape)

(22111, 10)


In [239]:
sig.head()

,rsid,chr,pos,ref,alt,minor_AF,beta,se,tstat,pval
0,rs159960,1,8476428,A,G,0.439602,0.004512,0.000763,5.91554,3.310750e-09
1,rs301805,1,8481016,T,G,0.414070,0.004470,0.000765,5.84052,5.208320e-09
2,rs301806,1,8482078,C,T,0.415936,0.004401,0.000764,5.75747,8.545140e-09
3,rs301807,1,8484823,A,G,0.414515,0.004475,0.000765,5.85053,4.904380e-09
4,rs301799,1,8489302,C,T,0.413459,0.004421,0.000766,5.77520,7.692560e-09


In [240]:
sumstat = sig.copy()
sumstat['cyto'] = None
for i, row in tqdm.tqdm(sumstat.iterrows(), total=sumstat.shape[0]):
    tmp = cytoBand.loc[(cytoBand[0]==f'chr{row.chr}') & (cytoBand[1]<=row.pos) & (cytoBand[2]>=row.pos)]
    sumstat.iloc[i,10] = f'{row.chr}{tmp[3].values[0]}'

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 22111/22111 [00:21<00:00, 1035.18it/s]


In [242]:
index_df = sumstat.loc[sumstat.groupby('cyto')['pval'].idxmin()].sort_values(by=['chr', 'pos']).reset_index(drop=True)
index_df = index_df.loc[index_df['rsid'].str.startswith('rs')]
print(index_df.shape)
index_df.to_csv('../blood_coloc/gwas/Asthma_ukb_indexsnp.csv', sep='\t', index=False)

(87, 11)


# Monocyte count

## GWAS preprocess

In [6]:
raw = pd.read_csv('../blood_coloc/gwas/30130_irnt.gwas.imputed_v3.both_sexes.tsv.bgz', compression='gzip', sep='\t')
print(raw.shape)
raw.head()

(13791467, 11)


,variant,minor_allele,minor_AF,low_confidence_variant,n_complete_samples,AC,ytx,beta,se,tstat,pval
0,1:15791:C:T,T,5.604550e-09,True,349856,0.003922,-0.004798,-355.405000,246.614000,-1.441140,0.149547
1,1:69487:G:A,A,5.935220e-06,True,349856,4.152940,-0.934619,-0.286586,0.483060,-0.593272,0.552999
2,1:69569:T:C,C,1.811110e-04,True,349856,126.725000,-7.005580,-0.050922,0.089648,-0.568024,0.570019
3,1:139853:C:T,T,5.845540e-06,True,349856,4.090200,-0.937214,-0.286084,0.483074,-0.592215,0.553707
4,1:692794:CA:C,C,1.106480e-01,False,349856,77422.100000,-289.390000,-0.003178,0.004064,-0.781944,0.434248


In [8]:
ukb_variants.head()

,variant,chr,pos,ref,alt,rsid,varid,consequence,consequence_category,info,...,p_hwe,n_called,n_not_called,n_hom_ref,n_het,n_hom_var,n_non_ref,r_heterozygosity,r_het_hom_var,r_expected_het_frequency
0,1:15791:C:T,1,15791,C,T,rs547522712,1:15791_C_T,splice_region_variant,missense,0.861678,...,0.500000,361194,0,361194,0,0,0,0.000000,NaN,0.000000
1,1:69487:G:A,1,69487,G,A,rs568226429,1:69487_G_A,missense_variant,missense,0.956975,...,0.500004,361194,0,361190,4,0,4,0.000011,NaN,0.000011
2,1:69569:T:C,1,69569,T,C,rs2531267,1:69569_T_C,missense_variant,missense,0.831664,...,0.506315,361194,0,361058,136,0,136,0.000377,NaN,0.000376
3,1:139853:C:T,1,139853,C,T,rs533633326,1:139853_C_T,splice_region_variant,missense,0.985255,...,0.500004,361194,0,361190,4,0,4,0.000011,NaN,0.000011
4,1:692794:CA:C,1,692794,CA,C,1:692794_CA_C,1:692794_CA_C,upstream_gene_variant,non_coding,0.824483,...,0.031437,361188,6,286823,70224,4141,74365,0.194425,16.9582,0.193734


In [9]:
merged = pd.merge(raw, ukb_variants[['variant', 'chr', 'pos', 'ref', 'alt', 'rsid']], on='variant', how='inner')
print(merged.shape)
merged = merged.loc[merged.chr.isin(np.arange(1,23))].astype({'chr': int})
print(merged.shape)
merged = merged.loc[merged.low_confidence_variant==False].sort_values(by=['chr', 'pos']).reset_index(drop=True)
print(merged.shape)
merged = merged.loc[merged['rsid'].str.startswith('rs')]
print(merged.shape)
merged.head()

(13791467, 16)
(13336576, 16)
(13135652, 16)
(12556367, 16)


,variant,minor_allele,minor_AF,low_confidence_variant,n_complete_samples,AC,ytx,beta,se,tstat,pval,chr,pos,ref,alt,rsid
1,1:693731:A:G,G,0.115822,False,349856,81042.30,-22.89850,0.001750,0.003841,0.455584,0.648689,1,693731,A,G,rs12238997
2,1:707522:G:C,C,0.097250,False,349856,68047.20,-8.87940,0.002788,0.004319,0.645537,0.518579,1,707522,G,C,rs371890604
3,1:717587:G:A,A,0.015718,False,349856,10998.10,-3.71433,-0.000652,0.010292,-0.063309,0.949521,1,717587,G,A,rs144155419
4,1:723329:A:T,T,0.001720,False,349856,1203.55,-59.28760,-0.058307,0.030551,-1.908550,0.056321,1,723329,A,T,rs189787166
5,1:730087:T:C,C,0.056479,False,349856,39519.00,-193.37800,-0.004384,0.005348,-0.819718,0.412377,1,730087,T,C,rs148120343


In [10]:
merged[['rsid', 'chr', 'pos', 'ref', 'alt', 'minor_AF', 'beta', 'se', 'tstat', 'pval']].to_csv('../blood_coloc/gwas/Monocyte_count_ukb.csv', sep='\t', index=False)

## Get index snps

In [11]:
raw = pd.read_csv('../blood_coloc/gwas/Monocyte_count_ukb.csv', sep='\t')
print(raw.shape)

(12556367, 10)


In [12]:
pval_thres = 5e-8
sig = raw.loc[raw['pval']<pval_thres].reset_index(drop=True)
print(sig.shape)

(73361, 10)


In [13]:
sumstat = sig.copy()
sumstat['cyto'] = None
for i, row in tqdm.tqdm(sumstat.iterrows(), total=sumstat.shape[0]):
    tmp = cytoBand.loc[(cytoBand[0]==f'chr{row.chr}') & (cytoBand[1]<=row.pos) & (cytoBand[2]>=row.pos)]
    sumstat.iloc[i,10] = f'{row.chr}{tmp[3].values[0]}'

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 73361/73361 [01:15<00:00, 972.48it/s]


In [14]:
print(sumstat.shape)
sumstat.head()

(73361, 11)


,rsid,chr,pos,ref,alt,minor_AF,beta,se,tstat,pval,cyto
0,rs75400926,1,2954378,T,C,0.149820,0.018653,0.003376,5.52532,3.291130e-08,1p36.32
1,rs201056490,1,2954379,T,C,0.149820,0.018653,0.003376,5.52532,3.291130e-08,1p36.32
2,rs147542464,1,2968454,C,CCCAT,0.147834,0.019039,0.003457,5.50733,3.645750e-08,1p36.32
3,rs717795,1,2970464,T,C,0.191292,0.016232,0.002955,5.49220,3.972350e-08,1p36.32
4,rs2981857,1,2971521,T,C,0.191477,0.016183,0.002948,5.48950,4.033510e-08,1p36.32


In [16]:
index_df = sumstat.loc[sumstat.groupby('cyto')['pval'].idxmin()].sort_values(by=['chr', 'pos']).reset_index(drop=True)
index_df = index_df.loc[index_df['rsid'].str.startswith('rs')]
print(index_df.shape)
index_df.to_csv('../blood_coloc/gwas/Monocyte_count_ukb_indexsnp.csv', sep='\t', index=False)

(315, 11)


# Neutrophil count

## GWAS preprocess

In [17]:
raw = pd.read_csv('../blood_coloc/gwas/30140_irnt.gwas.imputed_v3.both_sexes.tsv.bgz', compression='gzip', sep='\t')
print(raw.shape)
raw.head()

(13791467, 11)


,variant,minor_allele,minor_AF,low_confidence_variant,n_complete_samples,AC,ytx,beta,se,tstat,pval
0,1:15791:C:T,T,5.604550e-09,True,349856,0.003922,-0.002612,-161.915000,253.286000,-0.639257,0.522656
1,1:69487:G:A,A,5.935220e-06,True,349856,4.152940,0.419874,0.090707,0.496127,0.182831,0.854931
2,1:69569:T:C,C,1.811110e-04,True,349856,126.725000,-9.552040,-0.067077,0.092073,-0.728518,0.466297
3,1:139853:C:T,T,5.845540e-06,True,349856,4.090200,0.402127,0.086197,0.496142,0.173735,0.862073
4,1:692794:CA:C,C,1.106480e-01,False,349856,77422.100000,-313.180000,-0.004435,0.004174,-1.062470,0.288024


In [18]:
merged = pd.merge(raw, ukb_variants[['variant', 'chr', 'pos', 'ref', 'alt', 'rsid']], on='variant', how='inner')
print(merged.shape)
merged = merged.loc[merged.chr.isin(np.arange(1,23))].astype({'chr': int})
print(merged.shape)
merged = merged.loc[merged.low_confidence_variant==False].sort_values(by=['chr', 'pos']).reset_index(drop=True)
print(merged.shape)
merged = merged.loc[merged['rsid'].str.startswith('rs')]
print(merged.shape)
merged.head()

(13791467, 16)
(13336576, 16)
(13135652, 16)
(12556367, 16)


,variant,minor_allele,minor_AF,low_confidence_variant,n_complete_samples,AC,ytx,beta,se,tstat,pval,chr,pos,ref,alt,rsid
1,1:693731:A:G,G,0.115822,False,349856,81042.30,-240.8180,-0.002518,0.003944,-0.638365,0.523236,1,693731,A,G,rs12238997
2,1:707522:G:C,C,0.097250,False,349856,68047.20,-226.9890,-0.002722,0.004436,-0.613615,0.539470,1,707522,G,C,rs371890604
3,1:717587:G:A,A,0.015718,False,349856,10998.10,-58.8910,-0.007782,0.010570,-0.736193,0.461614,1,717587,G,A,rs144155419
4,1:723329:A:T,T,0.001720,False,349856,1203.55,16.0111,0.018672,0.031377,0.595068,0.551798,1,723329,A,T,rs189787166
5,1:730087:T:C,C,0.056479,False,349856,39519.00,-28.0723,0.000388,0.005493,0.070610,0.943708,1,730087,T,C,rs148120343


In [19]:
merged[['rsid', 'chr', 'pos', 'ref', 'alt', 'minor_AF', 'beta', 'se', 'tstat', 'pval']].to_csv('../blood_coloc/gwas/Neutrophil_count_ukb.csv', sep='\t', index=False)

## Get index snps

In [20]:
raw = pd.read_csv('../blood_coloc/gwas/Neutrophil_count_ukb.csv', sep='\t')
print(raw.shape)

(12556367, 10)


In [21]:
pval_thres = 5e-8
sig = raw.loc[raw['pval']<pval_thres].reset_index(drop=True)
print(sig.shape)

(65847, 10)


In [22]:
sumstat = sig.copy()
sumstat['cyto'] = None
for i, row in tqdm.tqdm(sumstat.iterrows(), total=sumstat.shape[0]):
    tmp = cytoBand.loc[(cytoBand[0]==f'chr{row.chr}') & (cytoBand[1]<=row.pos) & (cytoBand[2]>=row.pos)]
    sumstat.iloc[i,10] = f'{row.chr}{tmp[3].values[0]}'

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 65847/65847 [01:39<00:00, 664.14it/s]


In [23]:
print(sumstat.shape)
sumstat.head()

(65847, 11)


,rsid,chr,pos,ref,alt,minor_AF,beta,se,tstat,pval,cyto
0,rs10864351,1,8409672,C,A,0.499331,0.014042,0.002390,5.87647,4.194870e-09,1p36.23
1,rs8627,1,8412935,C,T,0.470298,0.014894,0.002387,6.23845,4.424440e-10,1p36.23
2,rs1889854,1,8415235,C,T,0.220550,0.017697,0.002883,6.13902,8.311910e-10,1p36.23
3,rs1889853,1,8415290,C,A,0.221524,0.017332,0.002877,6.02460,1.696930e-09,1p36.23
4,rs2784735,1,8421092,C,T,0.459481,0.014153,0.002403,5.89045,3.854950e-09,1p36.23


In [24]:
index_df = sumstat.loc[sumstat.groupby('cyto')['pval'].idxmin()].sort_values(by=['chr', 'pos']).reset_index(drop=True)
index_df = index_df.loc[index_df['rsid'].str.startswith('rs')]
print(index_df.shape)
index_df.to_csv('../blood_coloc/gwas/Neutrophil_count_ukb_indexsnp.csv', sep='\t', index=False)

(285, 11)


# IBD

## GWAS preprocess

In [246]:
raw = pd.read_csv('../blood_coloc/gwas/ibd_build37_59957_20161107.txt.gz', sep='\t', compression='gzip')
raw.shape

(9735446, 15)

In [ ]:
raw[['CHR-BP', 'ref', 'alt']] = raw.MarkerName.str.split('_', expand=True)
raw[['CHR', 'BP']] = raw['CHR-BP'].str.split(':', expand=True)

In [250]:
raw.head()

,MarkerName,Allele1,Allele2,Effect,StdErr,P.value,Direction,HetISq,HetChiSq,HetDf,HetPVal,Pval_IBDseq,Pval_IIBDGC,Pval_GWAS3,Min_single_cohort_pval,CHR-BP,ref,alt,CHR,BP
0,1:100000012_G_T,t,g,-0.0078,0.0147,0.5950,+-+,82.7,11.592,2,0.00304,0.239590,0.2746,0.002291,0.002291,1:100000012,G,T,1,100000012
1,1:10000006_G_A,a,g,0.0155,0.1109,0.8886,+--,54.3,4.373,2,0.11230,0.054628,0.9403,0.404895,0.054628,1:10000006,G,A,1,10000006
2,1:100000827_C_T,t,c,-0.0144,0.0136,0.2915,+-+,72.4,7.253,2,0.02661,0.499568,0.6470,0.005517,0.005517,1:100000827,C,T,1,100000827
3,1:100000843_T_C,t,c,0.0374,0.0289,0.1954,+++,0.0,0.575,2,0.75030,0.387836,0.2238,0.869664,0.223800,1:100000843,T,C,1,100000843
4,1:100000989_A_ATC,a,atc,0.0431,0.0300,0.1499,+++,0.0,0.835,2,0.65860,0.290844,0.1833,0.889055,0.183300,1:100000989,A,ATC,1,100000989


In [251]:
ref.head()

,SNP,CHR,BP,MAF,A1,A2
0,rs201725126,1,13116,0.186879,T,G
1,rs200579949,1,13118,0.186879,A,G
2,rs75454623,1,14930,0.479125,A,G
3,rs199856693,1,14933,0.050696,G,A
4,rs78601809,1,15211,0.268390,T,G


In [254]:
raw = raw.astype({'CHR': int, 'BP': int})

In [256]:
merged = pd.merge(raw, ref[['SNP', 'CHR', 'BP', 'A1', 'A2']], on=['CHR', 'BP'], how='inner')
merged.shape

(8404358, 23)

In [257]:
merged = merged.loc[((merged.ref==merged.A1) & (merged.alt==merged.A2)) | ((merged.ref==merged.A2) & (merged.alt==merged.A1))]
merged.shape

(8404202, 23)

In [260]:
merged.loc[merged.SNP=='rs6740847'] # effect on Allele2

,MarkerName,Allele1,Allele2,Effect,StdErr,P.value,Direction,HetISq,HetChiSq,HetDf,...,Pval_GWAS3,Min_single_cohort_pval,CHR-BP,ref,alt,CHR,BP,SNP,A1,A2
922366,2:182308352_A_G,a,g,-0.0924,0.0125,1.218000e-13,---,82.5,11.403,2,...,2.016410e-10,2.016410e-10,2:182308352,A,G,2,182308352,rs6740847,A,G


In [261]:
merged.Allele1 = merged.Allele1.str.upper()
merged.Allele2 = merged.Allele2.str.upper()

In [262]:
merged = merged.sort_values(by=['CHR', 'BP']).reset_index(drop=True)
merged.shape

(8404202, 23)

In [266]:
merged[['CHR', 'BP', 'SNP', 'Allele1', 'Allele2', 'Effect', 'StdErr', 'P.value']].to_csv('../blood_coloc/gwas/IBD_2016.csv', sep='\t', index=False)

## Get index snps

In [267]:
raw = pd.read_csv('../blood_coloc/gwas/IBD_2016.csv', sep='\t')
raw.shape

(8404202, 8)

In [268]:
pval_thres = 5e-8
sig = raw.loc[raw['P.value']<pval_thres].reset_index(drop=True)
print(sig.shape)

(12331, 8)


In [269]:
sig.head()

,CHR,BP,SNP,Allele1,Allele2,Effect,StdErr,P.value
0,1,1194804,rs11804831,T,C,0.0908,0.0164,3.312000e-08
1,1,1215424,rs1268339,T,C,0.0907,0.0163,2.752000e-08
2,1,7918904,rs4908704,A,G,-0.0902,0.0164,3.684000e-08
3,1,7919363,rs17374781,T,C,0.0898,0.0164,4.487000e-08
4,1,7919825,rs12745962,T,C,0.0919,0.0164,2.137000e-08


In [270]:
sumstat = sig.copy()
sumstat['cyto'] = None
for i, row in tqdm.tqdm(sumstat.iterrows(), total=sumstat.shape[0]):
    tmp = cytoBand.loc[(cytoBand[0]==f'chr{row.CHR}') & (cytoBand[1]<=row.BP) & (cytoBand[2]>=row.BP)]
    sumstat.iloc[i,8] = f'{row.CHR}{tmp[3].values[0]}'

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12331/12331 [00:12<00:00, 1004.20it/s]


In [284]:
index_df = sumstat.loc[sumstat.groupby('cyto')['P.value'].idxmin()].sort_values(by=['CHR', 'BP']).reset_index(drop=True)
print(index_df.shape)
index_df.to_csv('../blood_coloc/gwas/IBD_2016_indexsnp.csv', sep='\t', index=False)

(121, 9)
